In [ ]:
# For the RAM issues

import gc
gc.collect()

3

In [ ]:
!pip install torch transformers datasets

In [ ]:
import os
import re
import pickle
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from torch.optim import AdamW
from torch.utils.data import DataLoader, TensorDataset
from datasets import Dataset
from tqdm import tqdm
from transformers import RobertaTokenizer, RobertaForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Preprocessing

In [ ]:
dataset_dir = '/content/gdrive/MyDrive/Essentials in Text and Speech Processing'
csv_path = os.path.join(dataset_dir, 'fake_job_postings.csv')
df = pd.read_csv(csv_path)

# Drop features we know are useless from statistical model preprocessing
df = df.drop(['job_id'], axis=1)
df = df.drop(['salary_range'], axis=1)
df = df.drop(['benefits'], axis=1)
df = df.drop(['department'], axis=1)

In [ ]:
# The 'location' feature has inconsistencies so we standardize the format by splitting the 'location' column into Country, State, and City
def split_location(location):
    # Split the location by commas
    parts = location.split(',')

    # Ensure the list has 3 elements (Country, State, City), fill missing with 'Unknown'
    while len(parts) < 3:
        parts.append('unknown')  # Add 'Unknown' for missing elements

    return parts[:3]  # In case there are more than 3 parts

# Make sure there are no missing values in 'location'
df['location'] = df['location'].fillna('unknown')

# Apply the function to the 'location' column
df[['country', 'state', 'city']] = df['location'].apply(lambda loc: split_location(loc)).apply(pd.Series)

# Now fill any remaining NaN values with 'unknown' just in case
df[['country', 'state', 'city']] = df[['country', 'state', 'city']].fillna('Unknown')

# Drop the location feature as it is useles now
df = df.drop(['location'], axis=1)

# Insert "unknown" in place of null values
df = df.fillna('unknown')

# Lowercasing
df = df.apply(lambda col: col.map(lambda x: x.lower() if isinstance(x, str) else x))

# Remove punctuation marks and special characters
df = df.apply(lambda col: col.map(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', x) if isinstance(x, str) else x))

x = df.drop(['fraudulent'], axis=1) # Features
y = df['fraudulent'] # Labels
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
# Load pre-trained RoBERTa tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Preprocess the text features (combine columns)
text_columns = ['title', 'company_profile', 'description', 'requirements', 'industry', 'function', 'required_experience', 'required_education', 'employment_type', 'city', 'state', 'country']

# Apply a function to combine these features into one column
x_train_text = x_train[text_columns].apply(lambda row: ' '.join(row), axis=1)
x_test_text = x_test[text_columns].apply(lambda row: ' '.join(row), axis=1)

# Function to tokenize and pad sequences to a fixed length
def tokenize_and_pad(texts, tokenizer, max_length=512):
    return tokenizer(texts.tolist(), truncation=True, padding=True, max_length=max_length, return_tensors='pt')

# Tokenize the text data for both train and test sets
train_encodings = tokenize_and_pad(x_train_text, tokenizer)
test_encodings = tokenize_and_pad(x_test_text, tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Compute class weights based on the training labels
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

# Update model's loss function to include class weights
criterion = nn.CrossEntropyLoss(weight=class_weights)

# Ensure that y_train and y_test are tensors and not pandas Series
y_train_tensor = torch.tensor(y_train.values).to(device)
y_test_tensor = torch.tensor(y_test.values).to(device)

# Custom dataset class with correct label handling
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]  # Make sure labels are correctly passed as tensors
        return item

    def __len__(self):
        return len(self.labels)

# Create the Dataset for train and test sets
train_dataset = CustomDataset(train_encodings, y_train_tensor)
test_dataset = CustomDataset(test_encodings, y_test_tensor)

# Create DataLoader for train and test sets
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)

In [ ]:
# Load the pre-trained RoBERTa model for sequence classification
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)  # binary classification (fraudulent vs non-fraudulent)

# Move the model to GPU if available
model.to(device)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [ ]:
# Set up optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)

# Modify the training loop to use the custom criterion
epochs = 3
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader):
        batch = {k: v.to(device) for k, v in batch.items()}  # Move to GPU
        optimizer.zero_grad()  # Reset gradients

        # Forward pass with the custom criterion
        outputs = model(**batch)  # Forward pass
        logits = outputs.logits
        loss = criterion(logits, batch['labels'])  # Compute the weighted loss
        total_loss += loss.item()

        loss.backward()  # Backward pass
        optimizer.step()  # Update model weights

    print(f'Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader)}')

  0%|          | 0/894 [00:00<?, ?it/s]<ipython-input-8-6d3bbd38f85e>:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


In [ ]:
# Evaluation loop
model.eval()
predictions = []
true_labels = []

for batch in tqdm(test_loader):
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():  # Disable gradient calculation for evaluation
        outputs = model(**batch)

    logits = outputs.logits
    preds = torch.argmax(logits, dim=1).cpu().numpy()  # Convert logits to predicted class labels
    predictions.extend(preds)
    true_labels.extend(batch['labels'].cpu().numpy())

In [ ]:
# Get accuracy and classification report
accuracy = accuracy_score(true_labels, predictions)
report = classification_report(true_labels, predictions)
conf_matrix = confusion_matrix(true_labels, predictions)

In [ ]:
output_dir = '/content/gdrive/MyDrive/Essentials in Text and Speech Processing/Results'

# Save the SVM model just in case
with open(os.path.join(output_dir, 'roberta_model.pkl'), 'wb') as model_file:
    pickle.dump(model, model_file)

# Save the evaluation results just in case
with open(os.path.join(output_dir, 'roberta_classification_report.txt'), 'w') as report_file:
    report_file.write("Classification Report:\n")
    report_file.write(report + "\n")

    report_file.write("Accuracy:\n")
    report_file.write(f"{accuracy:.4f}\n\n")

    report_file.write("Confusion Matrix:\n")
    for row in conf_matrix:
        report_file.write(" ".join(map(str, row)) + "\n")

print("Model and classification report have been saved.")